In [1]:
import sys
import os
from pathlib import Path
# This appends the directory one level up (the root of your project) to the sys.path.
# Modify the path depending on the location of modules you want to import.
sys.path.append(os.path.abspath('../'))

In [2]:
from config.config_managers import DashboardConfigManager
from dataManager import DataManager



2024-11-21 13:52:31 - INFO - PyTorch version 2.2.2 available.


In [3]:
CONFIG_PATH = Path("/Users/ay227/Desktop/Final-Year/Thesis-Experiments/Online-Dashboard-Phase/dashboard-config.yaml")
config_manager = DashboardConfigManager(CONFIG_PATH)
dev_config = config_manager.development_config    

In [4]:

from dash import Dash, dcc, html, Output, Input, State
app = Dash(__name__, suppress_callback_exceptions=True)

app_config = config_manager.app_config
server = app.server  # Flask server instance for caching
variants_data = None

data_manager = DataManager(config_manager, server)

In [5]:
data_manager.load_data()

2024-11-21 13:52:33 - WARNING - Resolved path does not exist, checking alternative paths: /Users/ay227/Desktop/Final-Year/Thesis-Experiments/Online-Dashboard-Phase/deformer-dashboard/notebooks/My Drive
2024-11-21 13:52:33 - INFO - Found Google Drive directory for account ahmed.younes.sam@gmail.com: /Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com
2024-11-21 13:52:33 - INFO - Loading Dashboard Data from  /Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Thesis-Experiments/Experiments/BaseLineExperiment/ANERCorp_CamelLab_arabertv02


  0%|          | 0/12 [00:00<?, ?it/s]

2024-11-21 13:52:37 - WARNING - Resolved path does not exist, checking alternative paths: /Users/ay227/Desktop/Final-Year/Thesis-Experiments/Online-Dashboard-Phase/deformer-dashboard/notebooks/My Drive
2024-11-21 13:52:37 - INFO - Found Google Drive directory for account ahmed.younes.sam@gmail.com: /Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com
2024-11-21 13:52:37 - INFO - Loading Dashboard Data from  /Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Thesis-Experiments/Experiments/BaseLineExperiment/conll2003_bert


  0%|          | 0/12 [00:00<?, ?it/s]

{'ANERCorp_CamelLab_arabertv02': DashboardData(analysis_data=       Sentence Ids  Token Positions     Words    Tokens Word Pieces  \
 0                 0                0     [CLS]     [CLS]       [CLS]   
 1                 0                1  الصالحية  الصالحية    الصالحية   
 2                 0                2    المفرق    المفرق      المفرق   
 3                 0                3         -         -           -   
 4                 0                4       غيث       غيث         غيث   
 ...             ...              ...       ...       ...         ...   
 29706           960               10    للوليد     ##ليد  للو, ##ليد   
 29707           960               11        بن        بن          بن   
 29708           960               12      طلال      طلال        طلال   
 29709           960               13         .         .           .   
 29710           960               14     [SEP]     [SEP]       [SEP]   
 
       Core Tokens True Labels Token Selector Id Pred Labels  

In [6]:
analysis_data = data_manager.variants_data['ANERCorp_CamelLab_arabertv02'].analysis_data

In [7]:
import json

file_name = '/Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Thesis-Experiments/Experiments/BaseLineExperiment/ANERCorp_CamelLab_arabertv02/fine_tuning/evaluation_metrics.json'
corpus_file = '/Users/ay227/Library/CloudStorage/GoogleDrive-ahmed.younes.sam@gmail.com/My Drive/Final Year Experiments/Thesis-Experiments/Experiments/ExperimentData/corpora.json'
with open(file_name, 'r') as file:
    entity_outputs = json.load(file)  # Use json.load() to read file, not json.loads()
    
with open(corpus_file, 'r') as file:
    corpora = json.load(file)  # Use json.load() to read file, not json.loads()


In [49]:
import logging
from abc import ABC, abstractmethod
from dataclasses import dataclass, field

import numpy as np
import pandas as pd
from seqeval.metrics import classification_report as seq_classification
from seqeval.metrics import f1_score as seq_f1
from seqeval.metrics import precision_score as seq_precision
from seqeval.metrics import recall_score as seq_recall
from sklearn.metrics import classification_report as skl_classification
from sklearn.metrics import f1_score as skl_f1
from sklearn.metrics import precision_score as skl_precision
from sklearn.metrics import recall_score as skl_recall
from torch import nn
from seqeval.scheme import IOB1, IOB2, IOE1, IOE2, IOBES, BILOU, auto_detect

VALID_SCHEMES = {
    'IOB1':IOB1,
    'IOB2':IOB2,
    'IOE1':IOE1,
    'IOE2':IOE2,
    'IOBES':IOBES,
    'BILOU':BILOU
    }
class EvaluationStrategy(ABC):
    def __init__(self, inv_map):
        self.inv_map = inv_map
        self.ignore_index = nn.CrossEntropyLoss().ignore_index

    def align_predictions(self, predictions, truth):
        predictions = np.argmax(predictions, axis=2)
        batch_size, seq_len = predictions.shape

        truth_list = [[] for _ in range(batch_size)]
        pred_list = [[] for _ in range(batch_size)]

        for i in range(batch_size):
            for j in range(seq_len):
                if truth[i, j] != self.ignore_index:
                    truth_list[i].append(self.inv_map[truth[i][j]])
                    pred_list[i].append(self.inv_map[predictions[i][j]])
                    
        if len(truth_list) != len(pred_list):
            raise ValueError("Aligned predictions and truth have mismatched lengths.")
        return truth_list, pred_list

    def create_classification_report(self, results):
        lines = []
        for line in results.strip().split("\n")[1:]:
            if line.strip():
                tokens = line.split()
                # Remove intermediate aggregation if exists (multi-class)
                if len(tokens) > 5:
                    del tokens[1]
                lines.append(tokens)
        report = pd.DataFrame(
            lines, columns=["Tag", "Precision", "Recall", "F1", "Support"]
        )
        return report

class TokenEvaluationStrategy(EvaluationStrategy):
    def compute_metrics(self, true_labels, predictions):
        try:
            truth_list, pred_list = self.align_predictions(predictions, true_labels)
        except:
            logging.info('The labels already aligned, proceed with evaluation')
            truth_list, pred_list = true_labels, predictions
        
        flat_truth = [item for sublist in truth_list for item in sublist]
        flat_preds = [item for sublist in pred_list for item in sublist]
        report = skl_classification(y_true=flat_truth, y_pred=flat_preds, digits=4)
        report = self.create_classification_report(report)
        cleaned_report = self.clean_report(report)
        return {
            "Precision": skl_precision(
                y_true=flat_truth, y_pred=flat_preds, average="macro"
            ),
            "Recall": skl_recall(
                y_true=flat_truth, y_pred=flat_preds, average="macro"
                ),
            "F1": skl_f1(
                y_true=flat_truth, y_pred=flat_preds, average="macro"
                ),
            "classification": cleaned_report,
            "output": {"y_true": flat_truth, "y_pred": flat_preds},
        }

    def clean_report(self, report):
        report = report.copy()
        mask = report["Tag"] == "accuracy"
        accuracy_row = report[mask]
        if not accuracy_row.empty:
            # Get the accuracy value
            accuracy_value = accuracy_row["Precision"].values[
                0
            ]  # Assuming accuracy is stored in the 'Precision' column
            accuracy_support = accuracy_row["Recall"].values[
                0
            ]  # Assuming accuracy is stored in the 'Precision' column

            # Set the precision, recall, and F1-score to the accuracy value
            report.loc[mask, "Precision"] = accuracy_value
            report.loc[mask, "Recall"] = accuracy_value
            report.loc[mask, "F1"] = accuracy_value
            report.loc[mask, "Support"] = accuracy_support

            # Rename the tag from 'accuracy' to 'accuracy/micro' for clarity
            report.loc[report["Tag"] == "accuracy", "Tag"] = "accuracy/micro"
        return report
    
class EntityEvaluationStrategy(EvaluationStrategy):
    def compute_metrics(self, true_labels, predictions, entity_config):
        scheme = entity_config.get('scheme')  # Default to 'none' if not specified

        # Check if the scheme is valid and not 'none'
        try:
            truth_list, pred_list = self.align_predictions(predictions, true_labels)
        except:
            logging.info('The labels already aligned, proceed with evaluation')
            truth_list, pred_list = true_labels, predictions
            
        strict_outputs = self._evaluate_strict(truth_list, pred_list, scheme)
        non_strict_outputs = self._evaluate_non_strict(truth_list, pred_list)
        
        return {
            "strict": strict_outputs,
            "non_strict": non_strict_outputs,
            "output": {"y_true": truth_list, "y_pred": pred_list}
        }
        
        
    
    def _evaluate_strict(self, truth_list, pred_list, scheme):
        
        if scheme is not None and scheme in VALID_SCHEMES:
            scheme_class = VALID_SCHEMES[scheme]
            report = seq_classification(
                    y_true=truth_list,
                    y_pred=pred_list,
                    digits=4,
                    mode='strict',
                    scheme=scheme_class,
                )
            precision = seq_precision(
                    y_true=truth_list, y_pred=pred_list, average="micro", mode='strict', scheme = scheme_class
                )
            recall = seq_recall(
                    y_true=truth_list, y_pred=pred_list, average="micro", mode='strict', scheme = scheme_class
                )
            f1 = seq_f1(
                    y_true=truth_list, y_pred=pred_list, average="micro", mode='strict', scheme = scheme_class
                )
            
        else:
            logging.info("The scheme is unspecified; seqeval will auto-detect the scheme.")
            report = seq_classification(
                    y_true=truth_list,
                    y_pred=pred_list,
                    digits=4,
                    mode='strict',
                )
            scheme_class = auto_detect(pred_list, False)
            precision = seq_precision(
                    y_true=truth_list, y_pred=pred_list, average="micro", mode='strict', scheme = scheme_class
                )
            recall = seq_recall(
                    y_true=truth_list, y_pred=pred_list, average="micro", mode='strict', scheme = scheme_class
                )
            f1 = seq_f1(
                    y_true=truth_list, y_pred=pred_list, average="micro", mode='strict', scheme = scheme_class
                )
        
        return {
            "Precision": precision,
            "Recall": recall,
            "F1": f1,
            "classification": self.create_classification_report(report),
                
            }
    
        
    def _evaluate_non_strict(self, truth_list, pred_list,):
        
        report = seq_classification(
                    y_true=truth_list,
                    y_pred=pred_list,
                    digits=4,
                )
        precision = seq_precision(
                y_true=truth_list, y_pred=pred_list, average="micro"
            )
        recall = seq_recall(
                    y_true=truth_list, y_pred=pred_list, average="micro"
                )
        f1 = seq_f1(
                y_true=truth_list, y_pred=pred_list, average="micro"
            )
        return {
            "Precision": precision,
            "Recall": recall,
            "F1": f1,
            "classification": self.create_classification_report(report),
                
            }



class Evaluation:
    def __init__(self, inv_map, y_true, y_pred, evaluation_config):
        self.truths = y_true
        self.predictions = y_pred
        self.evaluation_config = evaluation_config
        self.token_strategy = TokenEvaluationStrategy(inv_map)
        self.entity_strategy = EntityEvaluationStrategy(inv_map)

    

    def evaluate(self):
        token_metrics = self.token_strategy.compute_metrics(
            self.truths, self.predictions
        )
        entity_metrics = self.entity_strategy.compute_metrics(
            self.truths, self.predictions, self.evaluation_config
        )

        # Combine or store results as needed
        return {"Token_Level": token_metrics, "Entity_Level": entity_metrics}

    def _prepare_results(self, metrics):
        results = pd.DataFrame.from_dict(self._round_and_slice(metrics))
        report = metrics["classification"]
        output = metrics["output"]
        return results, report, output
    
    def _prepare_entity_results(self, metrics):
        strict = metrics["strict"]
        non_strict = metrics["non_strict"]
        entity_strict_results = pd.DataFrame.from_dict(self._round_and_slice(strict))
        entity_non_strict_results = pd.DataFrame.from_dict(self._round_and_slice(non_strict))
        entity_strict_report = strict['classification']
        entity_non_strict_report = non_strict['classification']
        output = metrics["output"]
        return {
            'entity_strict_results': entity_strict_results,
            'entity_non_strict_results': entity_non_strict_results,
            'entity_strict_report': entity_strict_report,
            'entity_non_strict_report': entity_non_strict_report,
            'output': output,
        }

    def _round_and_slice(self, dictionary):
        # Slicing and rounding results for cleaner presentation
        
        keys_for_slicing = ["Precision", "Recall", "F1"]
        sliced_dict = {key: [round(dictionary[key], 4)] for key in keys_for_slicing}
        return sliced_dict
    
    def generate_results(self):
        metrics = self.evaluate()
        token_results, token_report, token_outputs = self._prepare_results(
            metrics["Token_Level"]
        )
        entity_level_outputs = self._prepare_entity_results(
            metrics["Entity_Level"]
        )
        
        

        return {
            "token_results": token_results,
            "token_report": token_report,
            "token_outputs": token_outputs,
            "entity_strict_results": entity_level_outputs['entity_strict_results'],
            "entity_non_strict_results": entity_level_outputs['entity_non_strict_results'],
            "entity_strict_report": entity_level_outputs['entity_strict_report'],
            "entity_non_strict_report": entity_level_outputs['entity_non_strict_report'],
            "entity_outputs": entity_level_outputs['output'],
        }


@dataclass
class Metrics:
    token_results: pd.DataFrame = field(default_factory=pd.DataFrame)
    token_report: pd.DataFrame = field(default_factory=pd.DataFrame)
    token_outputs: dict = field(default_factory=dict)
    entity_strict_results: pd.DataFrame = field(default_factory=pd.DataFrame)
    entity_non_strict_results: pd.DataFrame = field(default_factory=pd.DataFrame)
    entity_strict_report: pd.DataFrame = field(default_factory=pd.DataFrame)
    entity_non_strict_report: pd.DataFrame = field(default_factory=pd.DataFrame)
    entity_outputs: dict = field(default_factory=dict)

    @staticmethod
    def from_dict(data: dict):
        """Create an instance from a dictionary."""
        required_keys = [
        "token_results", "token_report", "token_outputs",
        "entity_strict_results", "entity_non_strict_results",
        "entity_strict_report", "entity_non_strict_report", "entity_outputs"
        ]
        missing_keys = [key for key in required_keys if key not in data]
        if missing_keys:
            raise ValueError(f"Missing required keys in data: {missing_keys}")
        return Metrics(**data)

    def to_dict(self):
        return {
            "token_results": self.token_results.to_dict(orient="records"),
            "token_report": self.token_report.to_dict(orient="records"),
            "token_outputs": self.token_outputs,
            "entity_strict_results": self.entity_strict_results.to_dict(orient="records"),     
            "entity_non_strict_results": self.entity_non_strict_results.to_dict(orient="records"),
            "entity_strict_report": self.entity_strict_report.to_dict(orient="records"), 
            "entity_non_strict_report": self.entity_non_strict_report.to_dict(orient="records"), 
            "entity_outputs": self.entity_outputs,
        }


In [50]:
y_true = entity_outputs['entity_outputs']['y_true']
y_pred = entity_outputs['entity_outputs']['y_pred']
corpus = corpora['ANERCorp_CamelLab']
labels_map = corpus["labels_map"]
inv_labels_map = {v: k for k, v in labels_map.items()}
evaluation_config = {
	'scheme': 'IOB2'
}


In [51]:
inv_labels_map

{0: 'O',
 1: 'B-PERS',
 2: 'I-PERS',
 3: 'B-ORG',
 4: 'I-ORG',
 5: 'B-LOC',
 6: 'I-LOC',
 7: 'B-MISC',
 8: 'I-MISC'}

In [52]:
evaluator = Evaluation(
                inv_labels_map, y_true, y_pred, evaluation_config
            )
results = evaluator.generate_results()
metrics = Metrics.from_dict(results)

2024-11-22 08:30:00 - INFO - The labels already aligned, proceed with evaluation
2024-11-22 08:30:00 - INFO - The labels already aligned, proceed with evaluation


In [54]:
metrics.entity_strict_results

,Precision,Recall,F1
0,0.8476,0.8299,0.8387


In [47]:
outs = evaluator.evaluate()


2024-11-22 08:26:19 - INFO - The labels already aligned, proceed with evaluation
2024-11-22 08:26:19 - INFO - The labels already aligned, proceed with evaluation


In [48]:

outs['Entity_Level']['strict']

{'Precision': (0.8475750577367206,),
 'Recall': (0.8299412030755314,),
 'F1': (0.8386654478976234,),
 'classification':         Tag Precision  Recall      F1 Support
 0       LOC    0.8927  0.9341  0.9129     668
 1      MISC    0.7720  0.6340  0.6963     235
 2       ORG    0.7842  0.7511  0.7673     450
 3      PERS    0.8599  0.8438  0.8518     858
 4     micro    0.8476  0.8299  0.8387    2211
 5     macro    0.8272  0.7908  0.8071    2211
 6  weighted    0.8451  0.8299  0.8365    2211}

In [9]:
from seqeval.metrics import f1_score, classification_report, f1_score
from seqeval.scheme import IOB2

print(classification_report(y_true, y_pred, mode='strict', digits=4))

f1_score(y_true, y_pred, mode='strict', scheme=IOB2,  average='micro')


              precision    recall  f1-score   support

         LOC     0.8927    0.9341    0.9129       668
        MISC     0.7720    0.6340    0.6963       235
         ORG     0.7842    0.7511    0.7673       450
        PERS     0.8599    0.8438    0.8518       858

   micro avg     0.8476    0.8299    0.8387      2211
   macro avg     0.8272    0.7908    0.8071      2211
weighted avg     0.8451    0.8299    0.8365      2211



0.8386654478976234

In [10]:
from seqeval.metrics import f1_score, classification_report

print(classification_report(y_true, y_pred, mode=None, digits=4))


              precision    recall  f1-score   support

         LOC     0.8919    0.9275    0.9094       676
        MISC     0.7366    0.6214    0.6741       243
         ORG     0.7630    0.7364    0.7494       459
        PERS     0.8835    0.8298    0.8558       905

   micro avg     0.8483    0.8178    0.8327      2283
   macro avg     0.8187    0.7788    0.7972      2283
weighted avg     0.8461    0.8178    0.8310      2283



In [11]:
from seqeval.metrics import f1_score

# Ground truth (true labels)
y_true = [
    ['O', 'B-PER', 'I-PER', 'O', 'B-LOC', 'O'],
    ['O', 'B-ORG', 'I-ORG', 'O']
]

# Predicted labels (with minor errors)
y_pred = [
    ['O', 'B-PER', 'O', 'O', 'B-LOC', 'O'],  # Misses I-PER
    ['O', 'B-ORG', 'O', 'O']                # Misses I-ORG
]

# Default mode
f1_default = f1_score(y_true, y_pred, average='micro', mode=None)

# Strict mode
f1_strict = f1_score(y_true, y_pred, average='micro', mode='strict')

print("Default Mode F1 Score:", f1_default)
print("Strict Mode F1 Score:", f1_strict)


Default Mode F1 Score: 0.3333333333333333
Strict Mode F1 Score: 0.3333333333333333


In [34]:
from seqeval.metrics import f1_score, classification_report
from seqeval.scheme import IOB1
# Define data
y_true = [['B-PER', 'I-PER', 'O', 'B-ORG', 'I-ORG', 'O']]
y_pred = [['B-PER', 'I-ORG', 'O', 'B-ORG', 'I-ORG', 'O']]

# Strict mode, no scheme
# f1_no_scheme_strict = f1_score(y_true, y_pred, average='micro', mode='strict', scheme=None)
f1_no_scheme_strict = classification_report(y_true, y_pred, mode='strict', zero_division='Warn')
no_strict = classification_report(y_true, y_pred, mode=None,)

print("Strict Mode with No Scheme F1 Score:", f1_no_scheme_strict)

print("No Strict Mode with F1 Score:", no_strict)




Strict Mode with No Scheme F1 Score:               precision    recall  f1-score   support

         ORG       1.00      1.00      1.00         1
         PER       0.00      0.00      0.00         1

   micro avg       0.50      0.50      0.50         2
   macro avg       0.50      0.50      0.50         2
weighted avg       0.50      0.50      0.50         2

No Strict Mode with F1 Score:               precision    recall  f1-score   support

         ORG       0.50      1.00      0.67         1
         PER       0.00      0.00      0.00         1

   micro avg       0.33      0.50      0.40         2
   macro avg       0.25      0.50      0.33         2
weighted avg       0.25      0.50      0.33         2



In [ ]:
from collections import defaultdict, Counter
from seqeval.scheme import IOB2, Tokens, auto_detect
from seqeval.metrics.sequence_labeling import get_entities
from seqeval.scheme import Entities


def flatten_strict_entities(entities):
    return [e.to_tuple()[1:] for sen in entities.entities for e in sen]

def calculate_confusion_matrix(y_true, y_pred):
    # Initialize confusion matrix data structure
    types = set([ent[0] for ent in y_true]).union([ent[0] for ent in y_pred])
    confusion_matrix = {typ: {'TP': 0, 'FP': 0, 'FN': 0} for typ in types}


    # Track matched predictions to avoid counting them more than once
    matched_pred_indices = set()

    # Check each true entity against predicted entities
    for true_ent in y_true:
        true_type, true_start, true_end = true_ent
        match_found = False

        for idx, pred_ent in enumerate(y_pred):
            pred_type, pred_start, pred_end = pred_ent

            if idx not in matched_pred_indices and true_type == pred_type and true_start == pred_start and true_end == pred_end:
                confusion_matrix[true_type]['TP'] += 1
                matched_pred_indices.add(idx)
                match_found = True
                break
        
        if not match_found:
            confusion_matrix[true_type]['FN'] += 1


    # Any unmatched prediction is a false positive
    for idx, pred_ent in enumerate(y_pred):
        if idx not in matched_pred_indices:
            pred_type = pred_ent[0]
            confusion_matrix[pred_type]['FP'] += 1

    return confusion_matrix



from collections import defaultdict, Counter

def compute_false_negatives(y_true, y_pred):
    fn_counts = defaultdict(Counter)
    true_indexed = {(t[1], t[2]): t[0] for t in y_true}  # Index true entities by boundaries
    pred_indexed = {(p[1], p[2]): p[0] for p in y_pred}  # Index predicted entities by boundaries

    # Iterate through true entities to find false negatives
    for (t_start, t_end), t_type in true_indexed.items():
        if (t_start, t_end) not in pred_indexed or pred_indexed[(t_start, t_end)] != t_type:
            # No matching prediction or type mismatch at the same position
            matched_type = pred_indexed.get((t_start, t_end), 'Boundary')
            fn_counts[t_type][matched_type] += 1

    return fn_counts

def compute_false_positives(y_true, y_pred):
    fp_counts = defaultdict(Counter)
    true_indexed = {(t[1], t[2]): t[0] for t in y_true}  # Index true entities by boundaries
    pred_indexed = {(p[1], p[2]): p[0] for p in y_pred}  # Index predicted entities by boundaries

    # Iterate through predicted entities to find false positives
    for (p_start, p_end), p_type in pred_indexed.items():
        if (p_start, p_end) not in true_indexed or true_indexed[(p_start, p_end)] != p_type:
            # No matching true entity or type mismatch at the same position
            matched_type = true_indexed.get((p_start, p_end), 'Boundary')
            fp_counts[p_type][matched_type] += 1

    return fp_counts



In [36]:
scheme = auto_detect(y_true, False)

entities_true = Entities(y_true, scheme, False)
entities_pred = Entities(y_pred, scheme, False)
true_entity_type = flatten_strict_entities(entities_true)
pred_entity_type = flatten_strict_entities(entities_pred)
# Example usage
# conf_matrix = calculate_confusion_matrix([e.to_tuple()[1:] for sen in entities_true.entities for e in sen], [e.to_tuple()[1:] for sen in entities_pred.entities for e in sen])



conf_matrix = calculate_confusion_matrix(true_entity_type, pred_entity_type)
print(conf_matrix)
fn_errors = compute_false_negatives(true_entity_type, pred_entity_type)
fp_errors = compute_false_positives(true_entity_type, pred_entity_type)

print("False Negatives:", dict(fn_errors))
print("False Positives:", dict(fp_errors))
1

{'PER': {'TP': 0, 'FP': 1, 'FN': 1}, 'ORG': {'TP': 1, 'FP': 0, 'FN': 0}}
False Negatives: {'PER': Counter({'Boundary': 1})}
False Positives: {'PER': Counter({'Boundary': 1})}


1

In [37]:
entity_y_true = get_entities(y_true)
entity_y_pred = get_entities(y_pred)

conf_matrix = calculate_confusion_matrix(entity_y_true, entity_y_pred)
print(conf_matrix)
fn_errors = compute_false_negatives(entity_y_true, entity_y_pred)
fp_errors = compute_false_positives(entity_y_true, entity_y_pred)

print("False Negatives:", dict(fn_errors))
print("False Positives:", dict(fp_errors))
1

{'PER': {'TP': 0, 'FP': 1, 'FN': 1}, 'ORG': {'TP': 1, 'FP': 1, 'FN': 0}}
False Negatives: {'PER': Counter({'Boundary': 1})}
False Positives: {'PER': Counter({'Boundary': 1}), 'ORG': Counter({'Boundary': 1})}


1

In [38]:
# Example usage
entity_y_true = get_entities(entity_outputs['entity_outputs']['y_true'])
entity_y_pred = get_entities(entity_outputs['entity_outputs']['y_pred'])


entities_true = Entities(entity_outputs['entity_outputs']['y_true'], scheme, False)
entities_pred = Entities(entity_outputs['entity_outputs']['y_pred'], scheme, False)



In [39]:
conf_matrix = calculate_confusion_matrix(entity_y_true, entity_y_pred)
print(conf_matrix)

{'LOC': {'TP': 627, 'FP': 76, 'FN': 49}, 'ORG': {'TP': 338, 'FP': 105, 'FN': 121}, 'PERS': {'TP': 751, 'FP': 99, 'FN': 154}, 'MISC': {'TP': 151, 'FP': 54, 'FN': 92}}


In [40]:
(627+151+751+338) / ((627+92)+(151+154)+(751+49)+(338+121))

0.8177836180464302

In [41]:
1867 / 2201

0.8482507950931395

In [43]:
conf_matrix = calculate_confusion_matrix(entity_y_true, entity_y_pred)
print(conf_matrix)

{'LOC': {'TP': 627, 'FP': 76, 'FN': 49}, 'ORG': {'TP': 338, 'FP': 105, 'FN': 121}, 'PERS': {'TP': 751, 'FP': 99, 'FN': 154}, 'MISC': {'TP': 151, 'FP': 54, 'FN': 92}}


In [45]:
total_metrics = {}
for metric in ['TP', 'FP', 'FN']:
    total_metrics[metric] = sum(details[metric] for details in conf_matrix.values())
print(total_metrics)

{'TP': 1867, 'FP': 334, 'FN': 416}


In [46]:
1867 / (1867+334)

0.8482507950931395

In [47]:


# Example usage
fn_errors = compute_false_negatives(entity_y_true, entity_y_pred)
fp_errors = compute_false_positives(entity_y_true, entity_y_pred)

print("False Negatives:", dict(fn_errors))
print("False Positives:", dict(fp_errors))
1

False Negatives: {'LOC': Counter({'Boundary': 42, 'MISC': 3, 'ORG': 3, 'PERS': 1}), 'PERS': Counter({'Boundary': 130, 'ORG': 13, 'LOC': 6, 'MISC': 5}), 'ORG': Counter({'Boundary': 84, 'LOC': 20, 'PERS': 11, 'MISC': 6}), 'MISC': Counter({'Boundary': 77, 'ORG': 9, 'LOC': 6})}
False Positives: {'LOC': Counter({'Boundary': 44, 'ORG': 20, 'MISC': 6, 'PERS': 6}), 'MISC': Counter({'Boundary': 40, 'ORG': 6, 'PERS': 5, 'LOC': 3}), 'ORG': Counter({'Boundary': 80, 'PERS': 13, 'MISC': 9, 'LOC': 3}), 'PERS': Counter({'Boundary': 87, 'ORG': 11, 'LOC': 1})}


1

In [312]:
import plotly.express as px
import pandas as pd

# Your original data
data = conf_matrix

# Prepare lists for DataFrame construction
actual = []
predicted = []
counts = []

for (act, pred), count in data.items():
    actual.append(act)
    predicted.append('None' if pred is None else pred)  # Replace None with 'None' for better visualization
    counts.append(count)

# Create DataFrame
df = pd.DataFrame({'Actual': actual, 'Predicted': predicted, 'Count': counts})

# Pivot to format suitable for heatmap
pivot_table = df.pivot(index='Actual', columns='Predicted', values='Count').fillna(0)

# Generate heatmap
fig = px.imshow(pivot_table,
                labels=dict(x="Predicted Entity Type", y="Actual Entity Type", color="Count"),
                x=pivot_table.columns,
                y=pivot_table.index,
                text_auto=True,
                aspect="auto")

fig.update_layout(
    title="Entity Recognition Confusion Matrix",
    xaxis_title="Predicted Entity Type",
    yaxis_title="Actual Entity Type"
)

fig.show()


In [331]:
errors

defaultdict(<function __main__.calculate_confusion_matrix.<locals>.<lambda>()>,
            {'PERS': defaultdict(int,
                         {'MISC': 10, 'ORG': 26, 'LOC': 12, 'Unmatched': 99}),
             'LOC': defaultdict(int,
                         {'PERS': 2, 'MISC': 6, 'ORG': 6, 'Unmatched': 76}),
             'MISC': defaultdict(int, {'LOC': 12, 'ORG': 18, 'Unmatched': 54}),
             'ORG': defaultdict(int,
                         {'MISC': 12,
                          'PERS': 22,
                          'LOC': 40,
                          'Unmatched': 105})})

In [256]:
import plotly.express as px
import pandas as pd

# Your original data
data = conf_matrix1

# Prepare lists for DataFrame construction
actual = []
predicted = []
counts = []

for (act, pred), count in data.items():
    actual.append(act)
    predicted.append('None' if pred is None else pred)  # Replace None with 'None' for better visualization
    counts.append(count)

# Create DataFrame
df = pd.DataFrame({'Actual': actual, 'Predicted': predicted, 'Count': counts})

# Pivot to format suitable for heatmap
pivot_table = df.pivot(index='Actual', columns='Predicted', values='Count').fillna(0)

# Generate heatmap
fig = px.imshow(pivot_table,
                labels=dict(x="Predicted Entity Type", y="Actual Entity Type", color="Count"),
                x=pivot_table.columns,
                y=pivot_table.index,
                text_auto=True,
                aspect="auto")

fig.update_layout(
    title="Entity Recognition Confusion Matrix",
    xaxis_title="Predicted Entity Type",
    yaxis_title="Actual Entity Type"
)

fig.show()


In [115]:
entity_tag = 'LOC'

false_negatives = set([e for e in entity_y_true if e[0] == entity_tag]) - set([e for e in entity_y_pred if e[0] == entity_tag])
for fn in false_negatives:
  t, fn_s, fn_e = fn
  for entity in entity_y_true:
      t_t, t_s, t_e = entity
      if fn_s == t_s or fn_e == t_e:
        if t_s!=t_e:
          print(entity)
id = 5594
for entity in entity_y_pred:
    t, s, e = entity
    if s == id:
        print(entity)

('LOC', 8786, 8787)
('LOC', 8864, 8865)
('LOC', 16466, 16467)
('LOC', 5593, 5594)
('LOC', 2545, 2546)
('LOC', 25446, 25447)
('LOC', 1615, 1616)
('LOC', 5594, 5594)


In [123]:
entity_tag = 'LOC'

false_postive = set([e for e in entity_y_pred if e[0] == entity_tag]) - set([e for e in entity_y_true if e[0] == entity_tag])

for fn in false_postive:
  t, fn_s, fn_e = fn
  for entity in entity_y_pred:
      t_t, t_s, t_e = entity
      if fn_s == t_s or fn_e == t_e:
        if t_s!=t_e:
          print(entity)


('LOC', 3989, 3990)
('LOC', 19016, 19017)
('LOC', 24575, 24576)
('LOC', 16963, 16965)
('LOC', 8989, 8990)
('LOC', 7007, 7008)
('LOC', 696, 697)
('LOC', 6445, 6447)
('LOC', 21117, 21118)
('LOC', 24615, 24616)


In [137]:
# so the false positive doesn't have to be false positive. 
id = 6445
for entity in entity_y_true:
    t, s, e = entity
    if s == id:
        print(entity)

('PERS', 6445, 6446)


('LOC', 5594, 5594)


In [104]:
pr =[
        tok for sen in entity_outputs['entity_outputs']['y_pred']
        for tok in sen
        ]


tr =[
        tok for sen in entity_outputs['entity_outputs']['y_true']
        for tok in sen
        ]

In [107]:
pr[8760:8790] == tr[8760:8790]

True

In [105]:
tr[8760:8790]

['O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-LOC',
 'I-LOC',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'B-LOC',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O',
 'O']

In [ ]:
('LOC', 8786, 8787)
('LOC', 8864, 8865)
('LOC', 16466, 16467)
('LOC', 5593, 5594)
('LOC', 2545, 2546)
('LOC', 25446, 25447)
('LOC', 1615, 1616)


In [59]:
entity_y_pred

[('LOC', 1, 1),
 ('PERS', 3, 4),
 ('PERS', 9, 10),
 ('LOC', 29, 29),
 ('LOC', 30, 30),
 ('LOC', 65, 65),
 ('LOC', 66, 66),
 ('LOC', 146, 146),
 ('LOC', 158, 158),
 ('LOC', 181, 181),
 ('PERS', 207, 208),
 ('LOC', 217, 217),
 ('LOC', 253, 253),
 ('PERS', 256, 256),
 ('LOC', 272, 272),
 ('ORG', 282, 282),
 ('ORG', 287, 287),
 ('LOC', 291, 291),
 ('PERS', 301, 301),
 ('PERS', 313, 313),
 ('LOC', 334, 335),
 ('LOC', 337, 337),
 ('LOC', 341, 341),
 ('PERS', 346, 346),
 ('PERS', 397, 397),
 ('LOC', 427, 427),
 ('PERS', 436, 436),
 ('LOC', 458, 458),
 ('LOC', 488, 488),
 ('ORG', 504, 504),
 ('PERS', 508, 508),
 ('LOC', 551, 551),
 ('LOC', 559, 559),
 ('ORG', 584, 584),
 ('LOC', 616, 617),
 ('ORG', 622, 622),
 ('PERS', 624, 626),
 ('LOC', 632, 632),
 ('ORG', 640, 640),
 ('LOC', 643, 643),
 ('LOC', 656, 656),
 ('LOC', 689, 689),
 ('LOC', 696, 697),
 ('ORG', 705, 705),
 ('LOC', 751, 751),
 ('PERS', 771, 772),
 ('PERS', 785, 786),
 ('LOC', 795, 795),
 ('LOC', 800, 801),
 ('LOC', 805, 805),
 ('LOC

In [58]:
entity_y_true

[('LOC', 0, 0),
 ('LOC', 1, 1),
 ('PERS', 3, 4),
 ('PERS', 9, 10),
 ('LOC', 29, 29),
 ('LOC', 30, 30),
 ('PERS', 44, 44),
 ('LOC', 65, 65),
 ('LOC', 66, 66),
 ('LOC', 158, 158),
 ('LOC', 181, 181),
 ('PERS', 207, 208),
 ('LOC', 217, 217),
 ('LOC', 253, 253),
 ('PERS', 256, 256),
 ('LOC', 272, 272),
 ('ORG', 282, 282),
 ('ORG', 287, 287),
 ('LOC', 291, 291),
 ('PERS', 301, 301),
 ('PERS', 313, 313),
 ('LOC', 334, 335),
 ('LOC', 337, 337),
 ('LOC', 341, 341),
 ('PERS', 346, 346),
 ('ORG', 378, 378),
 ('PERS', 397, 397),
 ('LOC', 427, 427),
 ('PERS', 436, 436),
 ('LOC', 458, 458),
 ('LOC', 488, 488),
 ('ORG', 504, 504),
 ('PERS', 508, 508),
 ('LOC', 551, 551),
 ('LOC', 559, 559),
 ('ORG', 584, 584),
 ('LOC', 616, 617),
 ('ORG', 622, 622),
 ('PERS', 624, 626),
 ('LOC', 632, 632),
 ('ORG', 640, 640),
 ('LOC', 643, 643),
 ('LOC', 656, 656),
 ('LOC', 689, 689),
 ('LOC', 696, 696),
 ('ORG', 697, 697),
 ('ORG', 705, 705),
 ('PERS', 745, 745),
 ('LOC', 751, 751),
 ('PERS', 771, 772),
 ('PERS', 7

# Debugging

In [197]:
ENTITY = 'LOC'
entity_false_negatives = {ENTITY: Counter()}
false_negatives = set([e for e in entity_y_true if e[0] == 'LOC']) - set([e for e in entity_y_pred if e[0] == 'LOC'])
for e in false_negatives:
    t_type, t_start, t_end = e
    for pred_ent in entity_y_pred:
        p_type, p_start, p_end = pred_ent
        if t_start == p_start and t_start == p_end:
            if p_type == 'LOC':
                print(pred_ent)
            entity_false_negatives[t_type][p_type]+=1
            

ENTITY = 'LOC'
entity_false_positives = {ENTITY: Counter()}
false_positive = set([e for e in entity_y_pred if e[0] == ENTITY]) - set([e for e in entity_y_true if e[0] == ENTITY]) 
for e in false_positive:
    p_type, p_start, p_end = e
    for true_ent in entity_y_true:
        t_type, t_start, t_end = true_ent
        if t_start == p_start and t_end == p_end:
            # if p_type == 'ORG':
            #     # if t_type == 'ORG':
            #         print(true_ent)
            if p_type == t_type:
                entity_false_positives[p_type][t_type]+=1

('LOC', 16466, 16466)
('LOC', 2545, 2545)
('LOC', 1615, 1615)
('LOC', 25446, 25446)
('LOC', 8786, 8786)


In [198]:
entity_false_positives

{'LOC': Counter()}

In [201]:
id = 8786
for entity in entity_y_true:
    t, s, e = entity
    if s == id:
        print(entity)
for entity in entity_y_pred:
    t, s, e = entity
    if s == id or e == id+1:
        print(entity)

('LOC', 8786, 8787)
('LOC', 8786, 8786)
('LOC', 8787, 8787)


In [203]:
for entity in false_negatives:
    t, s, e = entity
    # if t == 'LOC':
    #     print(entity)
    if s == 8786:
        print(entity)

('LOC', 8786, 8787)


In [196]:
for entity in false_positive:
    t, s, e = entity
   
    if s == 16466 or e == 16467:
        print(entity)

('LOC', 16466, 16466)
('LOC', 16467, 16467)


In [ ]:
for entity in entity_y_true:
    t, s, e = entity
    if s == 16963:
        print(entity)

In [ ]:
for entity in entity_y_pred:
    t, s, e = entity
    if s == 16963:
        print(entity)

('LOC', 16963, 16965)


In [ ]:
entity_y_true